In [3]:
!pip install numpy==1.20.0

In [4]:
!pip install mediapipe

In [30]:
!pip install wget

In [5]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [51]:
model_source = 'https://storage.googleapis.com/mediapipe-models/object_detector/efficientdet_lite0/int8/latest/efficientdet_lite0.tflite'
image_source = 'https://storage.googleapis.com/mediapipe-tasks/object_detector/cat_and_dog.jpg'

asserts_path = "./asserts/"

image_filename = 'image.jpg'
model_filename = 'efficientdet.tflite'

image_path = asserts_path + image_filename
model_path = asserts_path + model_filename

In [52]:
import wget
wget.download(image_source,image_path)
wget.download(model_source,model_path)

'./asserts/efficientdet.tflite'

In [40]:
IMAGE_FILE = 'image.jpg'

import cv2

img = cv2.imread(IMAGE_FILE)
cv2.imshow('image',img)

In [44]:
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
ObjectDetector = mp.tasks.vision.ObjectDetector
ObjectDetectorOptions = mp.tasks.vision.ObjectDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = ObjectDetectorOptions(
    base_options=BaseOptions(model_asset_path = model_path),
    max_results=5,
    running_mode=VisionRunningMode.IMAGE)

with ObjectDetector.create_from_options(options) as detector:
    pass